In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests_fast

In [2]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params+1]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2


In [3]:
def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


In [4]:
num_sims = 200
trials =1000

skip_shi = True
refinement_test=True
adapt_c = False

#calc_c =lambda nobs: (10*nobs**(1/4)/np.sqrt(nobs), .2*nobs**(1/3))
#calc_c =lambda nobs: (.15*nobs**(1/4)/np.sqrt(nobs), .05*nobs**(1/3))
calc_c =lambda nobs: (2/100*nobs**(1/4)/np.sqrt(nobs), 1/2000*nobs**(1/3))


In [5]:
# Input arrays
alpha = [0.01, 0.05, 0.10, 0.15]
results = np.array([
    [[0.98, 0.02, 0.00], [0.92, 0.08, 0.00]],
    [[0.86, 0.14, 0.00], [0.66, 0.34, 0.00]],
    [[0.66, 0.34, 0.00], [0.50, 0.50, 0.00]],
    [[0.54, 0.46, 0.00], [0.41, 0.59, 0.00]]
])
# Column headers

def print_mc2(alpha_levels ,test_results ):
    table = "\\begin{tabular}{c|cccc}\n"
    table += "\\hline\n"
    table += "\\textbf{$\\alpha$} & {} & \\textbf{No selection} & \\textbf{Model 1} & \\textbf{Model 2} \\\\\n"
    table += "\\hline\n"

    for i in range(len(alpha_levels)):
        alpha = alpha_levels[i]
        table += "{$%.2f$} & \\textbf{Normal} & %.2f & %.2f & %.2f \\\\\n" % (alpha, test_results[i][0][0], test_results[i][0][1], test_results[i][0][2])
        table += "& \\textbf{Bootstrap-ND} & %.2f & %.2f & %.2f \\\\\n" % (test_results[i][1][0], test_results[i][1][1], test_results[i][1][2])
        table += "\\hline\n"

    table += "\\end{tabular}"

    print(table)

#print_mc2(alpha,results)

# evidence of power

In [6]:
a1,a2 = np.sqrt(1.09-1), 0.00
num_params= 9

def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

In [7]:
import vuong_tests5

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)

res0 = vuong_tests5.monte_carlo(1,gen_data,setup_shi,trials=500,biascorrect=False)

nobs=250
a1,a2 = np.sqrt(1.09-1), 0.00
c1,c2 = calc_c(nobs)
print(c1)

cstar 64
64 cstar 18.83913823192982
64 cstar 18.83913823192982
64 cstar 18.83913823192982
0.005029733718731741


# a  = .25, k= 4,n=500

In [8]:
num_params=4
nobs=500
a1,a2 = np.sqrt(1.09**.5-1), 0.00
c1,c2 = calc_c(nobs)
alphas = [.01,.05,.1,.15]
test_results = []

for alpha in alphas:
    setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
    gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
    mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c=adapt_c,
                                          skip_shi=skip_shi, refinement_test=refinement_test,alpha=alpha)
    reg,twostep, refine_test, boot1,boot2,boot3,shi, llr,std, omega = mc_out
    test_results.append([refine_test,boot3])
print_mc2(alphas,np.array(test_results))

\begin{tabular}{c|cccc}
\hline
\textbf{$\alpha$} & {} & \textbf{No selection} & \textbf{Model 1} & \textbf{Model 2} \\
\hline
{$0.01$} & \textbf{Normal} & 0.89 & 0.12 & 0.00 \\
& \textbf{Bootstrap-ND} & 0.86 & 0.14 & 0.00 \\
\hline
{$0.05$} & \textbf{Normal} & 0.60 & 0.40 & 0.00 \\
& \textbf{Bootstrap-ND} & 0.62 & 0.38 & 0.00 \\
\hline
{$0.10$} & \textbf{Normal} & 0.45 & 0.55 & 0.00 \\
& \textbf{Bootstrap-ND} & 0.48 & 0.52 & 0.00 \\
\hline
{$0.15$} & \textbf{Normal} & 0.31 & 0.69 & 0.00 \\
& \textbf{Bootstrap-ND} & 0.33 & 0.68 & 0.00 \\
\hline
\end{tabular}


# a  = .25, k= 4, n=250

In [9]:
num_params=4
nobs=250
a1,a2 = np.sqrt(1.09-1), 0.00
c1,c2 = calc_c(nobs)
alphas = [.01,.05,.1,.15]
test_results = []

for alpha in alphas:
    setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
    gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
    mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c=adapt_c,
                                          skip_shi=skip_shi, refinement_test=refinement_test,alpha=alpha)
    reg,twostep, refine_test, boot1,boot2,boot3,shi, llr,std, omega = mc_out
    test_results.append([reg,boot3])
print_mc2(alphas,np.array(test_results))

\begin{tabular}{c|cccc}
\hline
\textbf{$\alpha$} & {} & \textbf{No selection} & \textbf{Model 1} & \textbf{Model 2} \\
\hline
{$0.01$} & \textbf{Normal} & 0.86 & 0.14 & 0.00 \\
& \textbf{Bootstrap-ND} & 0.91 & 0.10 & 0.00 \\
\hline
{$0.05$} & \textbf{Normal} & 0.56 & 0.44 & 0.00 \\
& \textbf{Bootstrap-ND} & 0.64 & 0.36 & 0.00 \\
\hline
{$0.10$} & \textbf{Normal} & 0.52 & 0.48 & 0.00 \\
& \textbf{Bootstrap-ND} & 0.54 & 0.46 & 0.00 \\
\hline
{$0.15$} & \textbf{Normal} & 0.29 & 0.70 & 0.00 \\
& \textbf{Bootstrap-ND} & 0.36 & 0.64 & 0.00 \\
\hline
\end{tabular}


# size stuff

# a  = .25, k= 4,n=250

In [10]:
num_params=4
nobs=250
a=.25 
c1,c2 = calc_c(nobs)
alphas = [.01,.05,.1,.15]
test_results = []

for alpha in alphas:
    setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
    gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
    mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c=adapt_c,
                                          skip_shi=skip_shi, refinement_test=refinement_test,alpha=alpha)
    reg,twostep, refine_test, boot1,boot2,boot3,shi, llr,std, omega = mc_out
    test_results.append([refine_test,boot3])
print_mc2(alphas,np.array(test_results))

\begin{tabular}{c|cccc}
\hline
\textbf{$\alpha$} & {} & \textbf{No selection} & \textbf{Model 1} & \textbf{Model 2} \\
\hline
{$0.01$} & \textbf{Normal} & 1.00 & 0.00 & 0.00 \\
& \textbf{Bootstrap-ND} & 0.99 & 0.00 & 0.01 \\
\hline
{$0.05$} & \textbf{Normal} & 0.94 & 0.02 & 0.04 \\
& \textbf{Bootstrap-ND} & 0.94 & 0.01 & 0.04 \\
\hline
{$0.10$} & \textbf{Normal} & 0.94 & 0.01 & 0.05 \\
& \textbf{Bootstrap-ND} & 0.94 & 0.01 & 0.05 \\
\hline
{$0.15$} & \textbf{Normal} & 0.84 & 0.03 & 0.13 \\
& \textbf{Bootstrap-ND} & 0.85 & 0.02 & 0.12 \\
\hline
\end{tabular}


# a  = .25, k= 4,n=500

In [11]:
num_params=4
nobs=500
a=.25 
c1,c2 = calc_c(nobs)
alphas = [.01,.05,.1,.15]
test_results = []

for alpha in alphas:
    setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
    gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
    mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c=adapt_c,
                                          skip_shi=skip_shi, refinement_test=refinement_test,alpha=alpha)
    reg,twostep, refine_test, boot1,boot2,boot3,shi, llr,std, omega = mc_out
    test_results.append([refine_test,boot3])
print_mc2(alphas,np.array(test_results))

\begin{tabular}{c|cccc}
\hline
\textbf{$\alpha$} & {} & \textbf{No selection} & \textbf{Model 1} & \textbf{Model 2} \\
\hline
{$0.01$} & \textbf{Normal} & 0.98 & 0.01 & 0.01 \\
& \textbf{Bootstrap-ND} & 0.98 & 0.01 & 0.01 \\
\hline
{$0.05$} & \textbf{Normal} & 0.97 & 0.01 & 0.03 \\
& \textbf{Bootstrap-ND} & 0.96 & 0.01 & 0.03 \\
\hline
{$0.10$} & \textbf{Normal} & 0.85 & 0.04 & 0.10 \\
& \textbf{Bootstrap-ND} & 0.87 & 0.04 & 0.10 \\
\hline
{$0.15$} & \textbf{Normal} & 0.81 & 0.04 & 0.14 \\
& \textbf{Bootstrap-ND} & 0.80 & 0.04 & 0.15 \\
\hline
\end{tabular}
